<a href="https://colab.research.google.com/github/uervitonsantos/Exercicios_Metodos_Quantitativos/blob/main/Avalia%C3%A7%C3%A3o_M%C3%A9todos_Quantitativo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Metodos Quantitativos

# 1. (1,0 ponto) A temperatura de deformação sob carga de duas formulações diferentes de tubos plásticos está sendo estudada. Duas amostras de 12 observações são preparadas usando cada formulação, as temperaturas de deformação (em °F) são relatadas a seguir:

a) Os dados apoiam a alegação de que a temperatura média de deformação sob carga para a formulação 1 é
maior que a da formulação 2? Use α = 0,05.

# 2. (1,0 ponto) O fabricante de um produto está investigando a resistência à ruptura de uma nova fibra sintética, que será utilizada para confecção de camisas de uso masculino. A resistência é geralmente afetada pela porcentagem de algodão usado na mistura de materiais para a fibra. O fabricante realizou um experimento com cinco níveis do conteúdo de algodão e reproduziu o experimento cinco vezes. Os dados estão apresentados na tabela a seguir.

a) Há evidências para apoiar a alegação de que a porcentagem de algodão afeta a média de resistência da nova
fibra? Use α = 0,05.

b) Se necessário, faça comparações entre os pares de médias. Que conclusões você pode tirar?

# 3. (2,0 pontos) A resistência à tração do cimento Portland está sendo estudada. Quatro diferentes técnicas de mistura foram usadas e os seguintes dados coletados:

a) Avalie se os pressupostos para a análise de variância são satisfeitos.

b) Teste a hipótese de que a técnica de mistura afeta a resistência do cimento. Use α = 0,05 e justifique a
escolha da técnica utilizada (ANOVA ou Kruskal-Wallis).

c) Se necessário, faça um teste de comparações múltiplas.

# 4. (1,0 ponto) Em um teste com um tipo de fusível, 9 peças foram testadas com 25% de sobrecarga. O tempo médio até queimar dessa amostra foi x 19, 2 minutos, com desvio-padrão populacional conhecido   2, 4 minutos. Sabendo que a população é normal calcule o intervalo de confiança 95% para a média μ da população.

# 5. (1,0 ponto) A eficácia da publicidade de duas marcas rivais de café foi comparada. Os participantes avaliaram
anúncios das duas marcas e classificaram a probabilidade geral de comprar o produto (de 0 a 10, com 10 sendo
"definitivamente vai comprar" o produto"). Metade dos participantes deu notas para um dos produtos, a outra metade
deu classificações para o outro produto.

Determine o teste a ser usado para verificar se existe diferença entre os dois grupos, realize o teste escolhido com 95% de confiança e discuta o significado dos resultados.

# 6. (1,0 ponto) Os dados a seguir indicam a soma de notas obtidas por um grupo de alunos, homens e mulheres, do curso de engenharia. Use o teste de Mann-Whitney para determinar se os valores diferem entre homens e mulheres.Investigue se os empates observados interferem no resultado.

# 7. (1,0 ponto) O batimento cardíaco de um grupo de sete alunos, em batimento por minuto (bpm), foi aferido antes do exercício (I), imediatamente após o exercício (II), e 5 minutos após o exercício (III). Use o teste estatístico de Friedman para avaliar as diferenças entre os valores nas três situações. Aplique o teste de comparações múltiplas não paramétrico, se necessário.

# 8. (1,0 ponto) Use o teste de comparações múltiplas da menor diferença significativa para avaliar os resultados do exercício a seguir. Antes, faça o teste de Kruskall-Wallis.

# 9. (1,0 ponto) Considere os dados abaixo sobre os 574 gols marcados nas 232 partidas em Copas do Mundo de futebol, desde a Copa do Uruguai em 1930 até a Copa do Japão/Coreia em 2002. Os dados foram organizados em classes que representam intervalos de tempo entre gols sucessivos. Utilizando o teste Qui-quadrado verifique aderência ao modelo de Exponencial.

Dica: a distribuição exponencial é usada para descrever intervalos de tempo entre eventos sucessivos e possui a
seguinte expressão: f(x) = λ*e-λx